# Overview
In this notebook we explore multithreading using python. We will first discuss relevant libraries before looking at a specific implimentation.

# Relevant Libraries
## The multiprocessing Module
The multiprocessing module is at the heart of python's parallelization capabilities. From this module we gain access to both multithreading and multiprocessing libraries and primitives. Additionally we get access to the synchronization primitives.

Below we outline the key objects required for most implimentations:

### Lock
The [Lock class](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Lock) is the python implimentation of a mutex for threads. Using the lock we can synchronize access to resources shared between threads.

In [28]:
# Create a lock
import multiprocessing
lock = multiprocessing.Lock()

# Wait until we get exclusive access to shared resource
lock.acquire() 

# Assign a value to a shared resource (pseudo code)
shared_int = 5

# Allow other threads to access shared resource and avoid deadlocks
lock.release()

We can also use the lock with a context manager and the with clause to simplify our code

In [1]:
# Create a lock
import multiprocessing
lock = multiprocessing.Lock()

with lock:
    shared_int = 5

The problem with this lock is that it is not a reentrant lock. This means that we cannot call acquire multiple times. The second time we will call acquire, we will block and cause a deadlock.

In [71]:
try:
    print("Getting lock ...")
    lock.acquire()
    print("Got lock, Getting lock again ...")
    lock.acquire(timeout=3)
    print("Gave up waiting after three seconds")
finally:
    lock.release()

Getting lock ...
Got lock, Getting lock again ...
Gave up waiting after three seconds


As a result we can either impliment our own logic to manage the lock state (bad idea) or use the RLock object from the threading module wich is reentrant.

### Shared Memory
Python provides two mechanisms for sharing memory between threads. 

#### Value
The [Value object](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) allows c_type objects to be shared between threads. For a list of the typecodes see [this article](https://docs.python.org/3/library/array.html#module-array). To create a shared integer for example, we would do the following:

In [33]:
# Create the shared integer
import multiprocessing
shared_int = multiprocessing.Value('i')

# Assign a value
shared_int.value = 5

In addition the Value object is also created with a lock property by default. Rather than manually managing our own locks, we can simply leverage the built in functionality as follows:

In [34]:
with shared_int.get_lock():
    shared_int.value += 1

In some cases, we may want multiple objects managed by the same Lock. In such an instance we can create the Value and tell the constructor not to create a Lock for it.

In [35]:
shared_int = multiprocessing.Value('i', lock=False)

We can also give it a pointer to an existing lock.

#### Array
The [Array object](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Array) is an array of Values. For example a character string is an array of characters.

In [32]:
import multiprocessing
shared_string = multiprocessing.Array('c', b'Hello, World!', lock=lock)
shared_string.value = b'new string'
print(shared_string.value)

b'new string'


## The threading Module
The threading module provides a number of low level utilities related to multithreading. In particular the Thread-Local data.

### Non-Shred Memory
In some cases we want to have duplicate variables which are private to each thread. For example, if a thread is a person, every person has a pair of shoes but each person has their own pair for their own use.

#### local
The [local object](https://docs.python.org/3/library/threading.html#thread-local-data) is the python implimentation of Thread-Static or Thread-Local data.

In [66]:
# Create instance of thread local memory
import threading
thread_local_data = threading.local()

# Store arbitrary data in this object
thread_local_data.x = 5
thread_local_data.y = 6

# Show our data is there
print("{0} {1}".format(thread_local_data.x, thread_local_data.y))

5 6


### Rlock
The [RLock object](https://docs.python.org/3/library/threading.html#rlock-objects) is a reentrant lock which will not block if the acquire function is called multiple times.

In [73]:
# Create a lock
import threading
lock = threading.RLock()

try:
    print("Getting lock ...")
    lock.acquire()
    print("Getting lock again ...")
    lock.acquire()
    print("Got lock again")
finally:
    lock.release()

Getting lock ...
Getting lock again ...
Got lock again


## The multiprocessing.pool Module
This module provides access to the ThreadPool and must be imported to gain access.


### ThreadPool
The [ThreadPool](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool)  controls a pool of worker threads. The ThreadPool is interface compatable with the Pool object used to manage processes. The ThreadPool provides several mechanisms to running code segments in parallel. We will only cover one method here; the *startmap()* method. Later we will see a non-blocking option.

In [50]:
import multiprocessing
import time

# Create a lock to synchronize access to STDOUT (the console)
lock = multiprocessing.Lock()

# Create a function to print to the console
def my_func(idx, lock):
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx
        
# Create a ThreadPool with three workers
my_pool = multiprocessing.pool.ThreadPool(processes=3)

# Create ten sets of parameters for the 10 functions we want to run in parallel
params = [(idx, lock) for idx in range(9)]

# Parallelize the functions, wait until they all complete, and assemble the results in an array coresponding to the input parameter order
apply_result = my_pool.starmap(my_func, params)

# Get the results in an array coresponding to the input parameter order
print(apply_result)

0 -> 0
1 -> 0
2 -> 0
0 -> 1
1 -> 1
2 -> 1
0 -> 2
1 -> 2
2 -> 2
3 -> 0
4 -> 0
5 -> 0
3 -> 1
4 -> 1
5 -> 1
3 -> 2
4 -> 2
5 -> 2
6 -> 0
7 -> 0
8 -> 0
6 -> 1
7 -> 1
8 -> 1
6 -> 2
7 -> 2
8 -> 2
[0, 1, 2, 3, 4, 5, 6, 7, 8]


It is important to note that the ThreadPool will halt at the first error. All reqults etc will be lost if an unhandled exception is encountered

In [49]:
import multiprocessing
import time

# Create a lock to synchronize access to STDOUT (the console)
lock = multiprocessing.Lock()

# Create a function to print to the console
def my_func(idx, lock):
    if idx == 4:
        raise Exception("This should have been caught")
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx
        
# Create a ThreadPool with three workers
my_pool = multiprocessing.pool.ThreadPool(processes=3)

# Create ten sets of parameters for the 10 functions we want to run in parallel
params = [(idx, lock) for idx in range(9)]

# Parallelize the functions, wait until they all complete, and assemble the results in an array coresponding to the input parameter order
apply_result = my_pool.starmap(my_func, params)

# Get the results in an array coresponding to the input parameter order
print(apply_result)

0 -> 0
1 -> 0
2 -> 0
0 -> 1
1 -> 1
2 -> 1
0 -> 2
1 -> 2
2 -> 2
3 -> 0
5 -> 0
6 -> 0
3 -> 1
6 -> 1
5 -> 1
3 -> 2
6 -> 2
5 -> 2
7 -> 0
8 -> 0
7 -> 1
8 -> 1
7 -> 2
8 -> 2


Exception: This should have been caught

We can see only one exception was raised and we dont know what parameters were responsible.

To remedy this problem, and others, we will impliment a more robust object.

# Example Implimentation
In this section we will extend the ThreadPool concept. We want a progress bar to visualize how much of the compute has finished at any given point in time. We also want to automagically capture the exceptions (if the user desires) so that the program does not hault and a graceful exit can be orchestrated.

In [93]:
import time
import multiprocessing
import multiprocessing.pool
from tqdm.notebook import tqdm

# This class will keep track of exceptions raised in multithreaded functions if the user
# has elected to do so when they create the EnhancedThreadPool

class EnhancedThreadPoolException(Exception):
    def __init__(self, function, parameters, root_exeption, message):
        self.functiom=function
        self.parameters=parameters
        self.__cause__=root_exeption
        super().__init__(message)

# This class will act as an enhanced threadpool
# Ideally i would extend the ThreadPool class but I do not have enough time to impliment 
# ALL the required functionality. As such it will remain a separate class until I can extend.

class EnhancedThreadPool():

    def __init__(self, num_workers=4, handle_exceptions=True):
        
        self.handle_exceptions = handle_exceptions
        
        # Create a pointer for a progress bar to track progress
        self.progress_bar = None
        
        # Create a lock to manage internal access to the progress bar
        self.lock = multiprocessing.Lock()
        
        # Create internal pool to manage processes
        self.pool = multiprocessing.pool.ThreadPool(processes=num_workers)
        
        # Create a shared memory object to store the number of completed threads
        # between the various threads running in parallel
        self.shared_memory_buffer = multiprocessing.Value('i', 0)

    def _create_progress_bar(self, num_ops):
        
        return tqdm(range(num_ops))
        
    def _update_progress_bar(self, i):
        
        self.progress_bar.last_print_n = i
        self.progress_bar.n = i
        self.progress_bar.set_description(f'Completed Ops')
        self.progress_bar.refresh()

    # To make things simple we will define an internal function to hide the details
    # of the progress bar. This way the user can wwrite functions without worrying
    # about updating or locking the progress bar.
    @staticmethod
    def _wrapper_function(func, args, kwargs, internal_params):
        
        # Extract internal arguments for the manager's internal functions
        internal_args, internal_kwargs = internal_params
        lock, shared_memory_buffer = internal_args
        handle_exceptions = internal_kwargs["handle_exceptions"]
        
        # Run the user specificed function and return the results
        # Handle the exceptions is the user has requested us to do so
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if handle_exceptions:
                return EnhancedThreadPoolException(func, (args,kwargs), e, "An error ocurred while running the parallel function")
            else:
                raise
        finally:
            # Update the buffer for the progress bar to indicate the function has completed
            lock.acquire()
            shared_memory_buffer.value = shared_memory_buffer.value + 1
            lock.release()
        
    # This function will run a set of functions in separate processes.
    # The main thread will block and update a progress bar as the functions complete
    # It will return a result set once all the processes have finished or an
    # error has been encountered.
    # This function will run a set of functions in separate processes.
    # The main thread will block and update a progress bar as the functions complete
    # It will return a result set once all the processes have finished or an
    # error has been encountered.
    def parralelize(self, func, arg_set, kwarg_set):
        
        # Setup vars to help kick off parallelization
        num_ops = len(arg_set)
        self.progress_bar = self._create_progress_bar(num_ops)
        self.shared_memory_buffer.value = 0
        
        # Configure the parameters for the parallelization
        func_set = [func for i in range(0, num_ops)]
        internal_param_set = [
            ([self.lock, self.shared_memory_buffer], 
             {"handle_exceptions": self.handle_exceptions}
            ) for i in range(0, num_ops)]   
        param_set = zip(func_set, arg_set, kwarg_set, internal_param_set)
        
        # Start the functions in separate parallel processes and don't block
        apply_result = self.pool.starmap_async(self._wrapper_function, param_set)
        
        # Wait for the proceses to finish while updating the progress bar
        while not apply_result.ready():
            self._update_progress_bar(self.shared_memory_buffer.value)
            time.sleep(0.5)
        self._update_progress_bar(self.shared_memory_buffer.value)

        # Return the results
        return apply_result.get()

Now that we have these base objects we can test them out.

In [96]:
import itertools

# Define the function we want to parallelize
def my_func(idx, lock):
    if idx == 4:
        raise Exception("This should have been caught")
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx

# Detine parameters for multithreading
num_ops = 10
lock = multiprocessing.Lock()
my_func_arg_set = list(zip(list(range(0, num_ops)), itertools.repeat(lock)))
my_func_kwarg_set = [{} for i in range(0, num_ops)]

# Start the process in parallel
etp = EnhancedThreadPool(num_workers=4)
results = etp.parralelize(my_func, my_func_arg_set, my_func_kwarg_set)
print(" ")
print(results)

  0%|          | 0/10 [00:00<?, ?it/s]

0 -> 0
1 -> 0
2 -> 0
3 -> 0
1 -> 1
2 -> 1
0 -> 1
3 -> 1
1 -> 2
2 -> 2
3 -> 2
0 -> 2
5 -> 0
6 -> 0
7 -> 0
8 -> 0
6 -> 1
5 -> 1
7 -> 1
8 -> 1
6 -> 2
5 -> 2
7 -> 2
8 -> 2
9 -> 0
9 -> 1
9 -> 2
 
[0, 1, 2, 3, EnhancedThreadPoolException('An error ocurred while running the parallel function'), 5, 6, 7, 8, 9]


We can see the exception did not blow things up and can be tied back to the original parameter set.